In [8]:
import face_recognition  # Importing face recognition library.
import os
import sys
import cv2  # Importing OpenCV library.
import numpy as np
import math
import pandas as pd
import smtplib
from email.message import EmailMessage


# Calculating confidence of recognition.
def recognition_confidence(face_distance, face_match_threshold=0.6):
    # Calculating the range between 1.0 and the face match threshold.
    range = (1.0 - face_match_threshold)
    # Computing a linear confidence value between 0 and 1 based on the face distance.
    linear_val = (1.0 - face_distance) / (range * 2.0)

    if face_distance > face_match_threshold:
        # If the face distance is greater than the face match threshold then return the linear value as a percentage.
        return str(round(linear_val * 100, 2)) + '%'
    else:
        # If the face distance is less than or equal to the face match threshold,
        # apply a non-linear transformation to the linear value
        # and return the result as a percentage
        value = (linear_val + ((1.0 - linear_val) * math.pow((linear_val - 0.5) * 2, 0.2))) * 100
        return str(round(value, 2)) + '%'


def email_alert(subject, body, to, image_path=None):
    # Creating a  function of email_alert.
    msg = EmailMessage()
    # Setting the content of the email message to the body argument.
    msg.set_content(body)
    # Setting the subject of the email message to the subject argument.
    msg['subject'] = subject
    # Setting the recipient of the email message to the argument.
    msg['to'] = to
    # Setting the sender of the email message to a hardcoded email address.
    user = "Add your email here (sender)"
    msg['from'] = user
    # Setting the password for the email account used to send the email message.
    password = "Add your password here(Given from Gmail).Remember to make security adjustments in Gmail account!!!"

    # If an image path is provided, then add the image as an attachment to the email message.
    if image_path:
        # Opening the image file and read its data.
        with open(image_path, 'rb') as f:
            image_data = f.read()
            # Setting the filename of the attachment to the basename of the image path.
            image_name = os.path.basename(image_path)
            # Adding the image data as an attachment to the email message.
            msg.add_attachment(image_data, maintype='image', subtype='jpeg', filename=image_name)

    # Creating an instance of the SMTP class with the Gmail SMTP server and port.
    server = smtplib.SMTP("smtp.gmail.com", 587)
    # Starting a TLS-encrypted connection to the SMTP server.
    server.starttls()
    # Log in to the SMTP server using the hardcoded email address and password.
    server.login(user, password)
    # Sending the email message.
    server.send_message(msg)
    # Closing down the SMTP connection.
    server.quit()


# Class  FaceRecognition with it definition and methods.
class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True

    def __init__(self):
        # Instantiating the methods.
        # Cal the encode_faces method to load the known faces from the "faces" directory and compute their encodings.
        self.encoding_faces()
        # Create a "recognised_faces" directory to store snapshots of recognised faces.
        self.create_recognised_faces_folder()
        # Create an "unknown_faces" directory to store snapshots of unrecognized faces.
        self.create_unknown_faces_folder()

    def encoding_faces(self):
        # Iterating over all the images in the "faces" directory.
        for image in os.listdir('faces'):
            # Loading the face image using face_recognition library.
            face_image = face_recognition.load_image_file(f"faces/{image}")
            # Computing the face encoding using face_recognition library.
            face_encoding = face_recognition.face_encodings(face_image)[0]

            # Adding the face encoding and name to the known face encodings and names lists, respectively.
            self.known_face_encodings.append(face_encoding)
            self.known_face_names.append(image)
        # Printing the list of known face names to the console.
        print(self.known_face_names)

    def create_recognised_faces_folder(self):
        # Creating a "recognised_faces" directory if it does not exist.
        if not os.path.exists('recognised_faces'):
            os.makedirs('recognised_faces')

    def create_unknown_faces_folder(self):
        # Creating an "unknown_faces" directory if it does not exist.
        if not os.path.exists('unknown_faces'):
            os.makedirs('unknown_faces')

    def run_face_recognition(self):
        # Opening the video capture device (webcam).
        video_capture = cv2.VideoCapture(0)

        # If the video capture device is not opened, exit the program.
        if not video_capture.isOpened():
            sys.exit('Video source not found...')

        while True:
            # Reading a frame from the video capture device.
            ret, frame = video_capture.read()

            # Processing every other frame of video to save time.
            if self.process_current_frame:
                # Resizing frame of video to 1/4 size for faster face recognition processing.
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

                # Converting the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses).
                rgb_small_frame = small_frame[:, :, ::-1]

                # Finding all the faces and face encodings in the current frame of video.
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                self.face_names = []
                for face_encoding in self.face_encodings:
                    # Comparing if the face is a match for the known face(s).
                    matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    name = "Unknown"
                    confidence = '???'

                    # Calculating the shortest distance between database picture to presented face.
                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)

                    best_match_index = np.argmin(face_distances)
                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = recognition_confidence(face_distances[best_match_index])

                        # Saving a snapshot of the recognized face in the "recognised_faces" folder.
                        cv2.imwrite(f'recognised_faces/{name}.jpg', frame)
                    else:
                        # Saving a snapshot of the unrecognized face in the "unknown_faces" folder.
                        stamp = pd.Timestamp.now().to_period('S')  # Adding time stamp
                        stamp.to_timestamp().strftime('%d-%m-%Y_%H-%M-%S')
                        c_time = stamp.to_timestamp().strftime(
                            '%d-%m-%Y_%H-%M-%S')  # Setting up date format(current time).
                        cv2.imwrite(f'unknown_faces/Unknown_face {c_time}.jpg', frame)
                        
                        # Remember to add recipient email in the next line of code (email_alert)!!!
                        email_alert("Warning", "Unknown face detected", "Add recipient email here !!!",
                                    image_path=f'unknown_faces/Unknown_face {c_time}.jpg')

                    self.face_names.append(f'{name} ({confidence})')

            self.process_current_frame = not self.process_current_frame

            # Displaying the result and percentage of confidence detection.
            for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                # Scaling up face locations in the frame (scaled to 1/4 size).
                top *= 3
                right *= 4
                bottom *= 4
                left *= 4

                # Detailing and displaying the frame with the name.
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)

            # Displaying the resulting image with frame.
            cv2.imshow('Face Recognition', frame)

            # Pressing 'q'  button on the keyboard to quit the program (breaking the infinite loop).
            if cv2.waitKey(1) == ord('q'):
                break

        # Releasing handle to the webcam.
        video_capture.release()
        cv2.destroyAllWindows()


# Running the face recognition application.
if __name__ == '__main__':
    fr = FaceRecognition()
    fr.run_face_recognition()


['Michal.jpeg', 'Rambo.jpeg', 'Malgorzata_Soltycz.jpeg', 'Tharusha.jpeg', 'Carl.jpeg', 'Alex_Soltycz.jpeg']
